In [2]:
import pandas as pd
import numpy as np
import sqlite3 as sql
conn = sql.connect("S30_ETL_Assignment.db")

tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql_query(tables_query, conn)
print(tables_df)



              name
0  sqlite_sequence
1            sales
2           orders
3            items
4        customers


In [3]:
customer_query = '''SELECT * FROM customers'''
cust_df = pd.read_sql_query(customer_query, conn)
customer_df = cust_df
# customer_df.set_index('customer_id', inplace=True)
customer_df.head(5)

,customer_id,age
0,1,38
1,2,43
2,3,13
3,4,39
4,5,18


In [4]:
sqlite_table = '''SELECT * FROM sqlite_sequence'''
df = pd.read_sql_query(sqlite_table, conn)
sqlite_table_df = df
sqlite_table_df.set_index('name', inplace=True)
sqlite_table_df.head(5)

,seq
name,
items,3
customers,100
sales,500
orders,1500


In [5]:
items_query = '''SELECT * FROM items'''
item_df = pd.read_sql_query(items_query, conn)
items_df = item_df
# items_df.set_index('item_id', inplace=True)
items_df.head()

,item_id,item_name
0,1,x
1,2,y
2,3,z


In [6]:
sales_query = '''SELECT * FROM sales'''
sale_df = pd.read_sql_query(sales_query, conn)
sales_df = sale_df
# sales_df.set_index('sales_id', inplace=True)
sales_df.head()


,sales_id,customer_id
0,1,10
1,2,85
2,3,42
3,4,93
4,5,75


In [7]:
cust_sales = pd.merge(customer_df, sales_df, on='customer_id',how='left')
cust_sales.head(20)

,customer_id,age,sales_id
0,1,38,110
1,1,38,285
2,1,38,464
3,1,38,473
4,2,43,75
5,2,43,163
6,2,43,255
7,2,43,303
8,2,43,455
9,3,13,172


In [8]:
order_query = '''SELECT * FROM orders'''
order_df = pd.read_sql_query(order_query, conn)
orders_df = order_df
# orders_df.set_index('order_id', inplace=True)
orders_df.head()

,order_id,sales_id,item_id,quantity
0,1,1,1,2.0
1,2,1,2,NaN
2,3,1,3,1.0
3,4,2,1,1.0
4,5,2,2,1.0


In [9]:
cust_sales_order = pd.merge(cust_sales, orders_df, on='sales_id', how='left')
consolidated_df = pd.merge(cust_sales_order, items_df, on='item_id', how='left')
consolidated_df.head(20)


,customer_id,age,sales_id,order_id,item_id,quantity,item_name
0,1,38,110,328,1,NaN,x
1,1,38,110,329,2,NaN,y
2,1,38,110,330,3,1.0,z
3,1,38,285,853,1,1.0,x
4,1,38,285,854,2,1.0,y
5,1,38,285,855,3,1.0,z
6,1,38,464,1390,1,1.0,x
7,1,38,464,1391,2,1.0,y
8,1,38,464,1392,3,1.0,z
9,1,38,473,1417,1,1.0,x


In [16]:
cons_filter_age =  consolidated_df[(consolidated_df['age'] >= 18) & (consolidated_df['age'] <= 35)]
cust_table = cons_filter_age.groupby(['customer_id']).sum().reset_index()
customer_quant = cust_table[['customer_id', 'quantity',]]
customer_quant['quantity'] = customer_quant['quantity'].astype(int)
customer_quant.head(5)
# customer_quant.to_csv('customer_quant.csv', index=False, sep=';')

C:\Users\marion bill garcia\AppData\Local\Temp\ipykernel_187020\2834371199.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_quant['quantity'] = customer_quant['quantity'].astype(int)


,customer_id,quantity
0,5,10
1,7,43
2,9,11
3,16,17
4,22,13


In [108]:
all_three = cons_filter_age.groupby(['customer_id','item_name','age']).sum().reset_index()
all_three['quantity'] = all_three['quantity'].astype(int)
all_three_quant = all_three[['customer_id', 'item_name', 'quantity']]
all_three_quant.head(5)
# all_three_quant.to_csv('all_three_quant.csv', index=False, sep=';')

,customer_id,item_name,quantity
0,5,x,3
1,5,y,3
2,5,z,4
3,7,x,14
4,7,y,19
